# Sesión #7


# Programa 

1. Unir bases de datos
    - Ejemplo con la ENOE
2. Aplicación: ENIGH 2020

In [1]:
# Importar librerías 
import pandas as pd
#import matplotlib.pyplot as plt
import numpy as np

# Unir bases de datos 
Los datos contenidos en los objetos de pandas se pueden combinar de varias formas:

- `pandas.merge` une filas en DataFrames basándose en una o más claves. Esto resultará familiar para los usuarios de SQL u otras bases de datos relacionales como la ENOE, ya que implementa operaciones de unión de bases de datos.
- `pandas.concat` concatena o “apila” objetos a lo largo de un eje.
- El método `combine_first` permite unir datos superpuestos para completar los valores perdidos en un objeto con valores de otro. 

A continuación veremos ejemplos de cada uno de estos. 

## Merge
Las operaciones de `merge` o `join` combinan conjuntos de datos vinculando filas con una o más claves. Estas operaciones son fundamentales para las bases de datos relacionales. La función de `merge` en pandas es el principal punto de entrada para usar estos algoritmos en los datos.

Primero, ejemplificaremos el uso de estas funciones con bases sencillas. Considere los siguientes DataFrame:

In [3]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'], 'data1': range(7)})
df2 = pd.DataFrame({'key': ['a', 'b', 'd'], 'data2': range(3)})
print(df1)
print(df2)

  key  data1
0   b      0
1   b      1
2   a      2
3   c      3
4   a      4
5   a      5
6   b      6
  key  data2
0   a      0
1   b      1
2   d      2


Suponga que deseamos unir ambas bases de tal forma que, para cada elemento del df1 que tenga correspondencia con un elemento del df2, le adjuntemos el elemento correspondiente del df2. 

Este es un ejemplo de combinación de varios-a-uno (*many-to-one*), pues los datos en df1 tienen varias filas etiquetadas como a y b, mientras que df2 tiene solo una fila para cada valor en la columna `key`. 

Al usar `merge` con estos objetos *debemos* indicar la variable sobre la cual se van a unir las bases: 

In [4]:
pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


Por defecto, `merge` hace un "inner join": las claves del DataFrame resultante son la intersección o el conjunto común que se encuentra en ambas tablas. Otras opciones de unión de bases se especifican con el argumento `how`: 

| Opción | Comportamiento |
| :--- | :--- |
| inner | Utiliza solo las combinaciones de *claves* observadas en ambas tablas |
| left | Utiliza todas las combinaciones de *claves* encontradas en la tabla de la izquierda |
| right | Utiliza todas las combinaciones de *claves* encontradas en la tabla de la derecha |
| outer | Utiliza todas las combinaciones de *claves* encontradas en ambas tablas |


In [5]:
# Por ejemplo:
pd.merge(df1, df2, how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


Las combinaciones de varios-a-varios (*many-to-many*) tienen un comportamiento bien definido, aunque no necesariamente intuitivo. Considere los siguientes DataFrame:

In [6]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'], 'data1': range(6)})
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'], 'data2': range(5)})
print(df1)
print(df2)

  key  data1
0   b      0
1   b      1
2   a      2
3   c      3
4   a      4
5   b      5
  key  data2
0   a      0
1   b      1
2   a      2
3   b      3
4   d      4


Many-to-many joins forma el producto Cartesiano de las filas. Como hay tres filas 'b' en el DataFrame izquierdo y dos en el derecho, hay seis filas 'b' en el resultado:

In [8]:
pd.merge(df1, df2, how='inner')

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


Para fusionar con varias claves al mismo tiempo, basta con usar una lista de nombres de columna en el argumento `on`:

In [9]:
# Considere los siguientes DaaFrame:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'], 
                     'key2': ['one', 'two', 'one'],
                     'lval': [1, 2, 3]})
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'], 
                      'key2': ['one', 'one', 'one', 'two'],
                      'rval': [4, 5, 6, 7]})
print(left)
print(right)

  key1 key2  lval
0  foo  one     1
1  foo  two     2
2  bar  one     3
  key1 key2  rval
0  foo  one     4
1  foo  one     5
2  bar  one     6
3  bar  two     7


In [10]:
# Resultado del outer join
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


Argumentos importantes de la función `merge`

| Argumento | Descripción |
| :--- | :--- |
| left | El DataFrame se unirá sobre el DataFrame del lado izquierdo. |
| right | El DataFrame se unirá sobre el DataFrame del lado derecho. | |
| how | Uno de los siguientes 'inner', 'outer', 'left', or 'right'; por defecto es 'inner'.
| on | Nombres de columnas a los que unirse. Debe encontrarse en ambos objetos DataFrame. Si no se especifica y no se proporcionan otras claves de combinación, se utilizará la intersección de los nombres de las columnas a la izquierda y a la derecha como claves de combinación. |
| left_on | Columnas en el DataFrame izquierdo para usar como claves de combinación. |
| right_on | Análogo a left_on. |
| left_index | Utilizar el índice de filas en el DataFrame de la izquierda como clave de combinación (o claves si es un índice múltiple). |
| right_index | Análogo a left_index. |
| suffixes | Tupla de strings para agregar a los nombres de las columnas en caso de superposición; por defecto es ('_x', '_y') (por ejemplo, si 'data' en ambos objetos DataFrame, aparecería como 'datos_x' y 'datos_y' en el resultado).| 

Los DataFrame de pandas tienen una instancia `join` conveniente para fusionar por índice. También se puede usar para combinar muchos objetos DataFrame que tienen índices iguales o similares pero columnas que no se superponen. En parte por razones heredadas (es decir, versiones mucho anteriores de pandas), el método de combinación de DataFrame realiza una combinación a la izquierda en las claves de combinación, conservando exactamente el índice de fila del marco izquierdo.

Sin embargo, es preferible dominar `merge` pues es más general que `join`. 

## Concatenación 
En el contexto de objetos pandas como Series y DataFrame, existen varías cosas en las que pensar al hacer la concatenación:

- Si los objetos están indexados de manera diferente en los otros ejes, ¿deberíamos combinar los elementos distintos en estos ejes o usar solo los valores compartidos (la intersección)?
- ¿Es necesario que los fragmentos de datos concatenados sean identificables en el objeto resultante?
- ¿El “eje de concatenación” contiene datos que deben conservarse? En muchos casos, las etiquetas enteras predeterminadas en un DataFrame se descartan mejor durante la concatenación.

La función `concat` en pandas proporciona una forma coherente de abordar cada una de estas preocupaciones.

In [3]:
# Considere las siguientes series
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])


Usar `concat` con estos objetos en una lista pega los valores y los índices:

In [4]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

Por defecto, `concat` funciona a lo largo del `axis=0`, produciendo otra Serie. Si utiliza `axis=1`, el resultado será un DataFrame:

In [5]:
pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


En este caso no hay superposición en el otro eje, que como puede ver es la unión ordenada (la *outer join*) de los índices. En su lugar, puede cruzarlos utilizando `join='inner'`. Por ejemplo: 

In [6]:
# Creamos una nueva Serie 
s4 = pd.concat([s1, s3])
s4

a    0
b    1
f    5
g    6
dtype: int64

In [7]:
# Veamos que ocurre cuando concatenamos las columnas
pd.concat([s1, s4], axis=1)

,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [8]:
# Cuando utilizamos el argumento join='inner' desaparecen las filas f y g
pd.concat([s1, s4], axis=1, join='inner')

,0,1
a,0,0
b,1,1


Un problema potencial es que las piezas concatenadas no son identificables en el resultado. En su lugar, suponga que desea crear un índice jerárquico en el eje de concatenación. Para hacer esto, use el argumento de `keys`. 

In [11]:
# Por ejemplo:
pd.concat([s1, s2, s3], keys=['one', 'two', 'three'])

one    a    0
       b    1
two    c    2
       d    3
       e    4
three  f    5
       g    6
dtype: int64

En el caso de combinar Series a lo largo del `axis=1`, las claves se convierten en el encabezado de la columna DataFrame:

In [12]:
pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three'])

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


La misma lógica se extiende a los objetos DataFrame. Considere los siguientes ejemplos:

In [14]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],
                   columns=['one', 'two'])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],
                   columns=['three', 'four'])
print(df1)
print(df2)

   one  two
a    0    1
b    2    3
c    4    5
   three  four
a      5     6
c      7     8


In [17]:
# Concatenar las filas
pd.concat([df1, df2], keys=['level1', 'level2'])

one  two  three  four
level1 a  0.0  1.0    NaN   NaN
       b  2.0  3.0    NaN   NaN
       c  4.0  5.0    NaN   NaN
level2 a  NaN  NaN    5.0   6.0
       c  NaN  NaN    7.0   8.0

In [18]:
# Concatenar las columnas 
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

También puede haber DataFrames en los que el índice de fila no contiene ningún dato relevante, en este caso se puede usar `ignore_index=True`. Por ejemplo:

In [4]:
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])
print(df1)
print(df2)

          a         b         c         d
0  0.191259 -0.744865  0.772615 -0.367835
1 -0.861065 -0.848704 -0.816014  0.856577
2 -0.734347 -1.229024 -0.002831 -0.716914
          b         d         a
0 -0.450678  0.258555  0.240904
1  0.387560 -0.118677  0.721182


In [7]:
# Concatenamos ignorando el índice
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,0.191259,-0.744865,0.772615,-0.367835
1,-0.861065,-0.848704,-0.816014,0.856577
2,-0.734347,-1.229024,-0.002831,-0.716914
3,0.240904,-0.450678,NaN,0.258555
4,0.721182,0.387560,NaN,-0.118677


## Combinar datos con superposición

Existe otra situación de combinación de datos que no se puede expresar como una operación de combinación o concatenación. Puede tener dos conjuntos de datos cuyos índices se superponen total o parcialmente. 

Como ejemplo, considere la función `where` de NumPy, que realiza el equivalente orientado a matrices de una expresión if-else:

In [8]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan], index=['f', 'e', 'd', 'c', 'b', 'a'])
b = pd.Series(np.arange(len(a), dtype=np.float64), index=['f', 'e', 'd', 'c', 'b', 'a'])
b[-1] = np.nan

print(a)
print(b)

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64
f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64


In [9]:
np.where(pd.isnull(a), b, a)

array([0. , 2.5, 2. , 3.5, 4.5, nan])

Los objetos Series tiene el método `combine_first`, que realiza el equivalente de esta operación junto con la lógica de alineación de datos habitual de pandas:

In [13]:
a.combine_first(b)

f    0.0
e    2.5
d    2.0
c    3.5
b    4.5
a    NaN
dtype: float64

Con DataFrames, `combine_first` hace lo mismo columna por columna, por lo que puede pensar en ello como "parchar" los datos faltantes en el primer objeto con datos del objeto dentro del método:

In [14]:
df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan], 'b': [np.nan, 2., np.nan, 6.], 
                    'c': range(2, 18, 4)})
df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.], 'b': [np.nan, 3., 4., 6., 8.]})
print(df1)
print(df2)

     a    b   c
0  1.0  NaN   2
1  NaN  2.0   6
2  5.0  NaN  10
3  NaN  6.0  14
     a    b
0  5.0  NaN
1  4.0  3.0
2  NaN  4.0
3  3.0  6.0
4  7.0  8.0


In [15]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


# ENOE
